# Part 1 pre-acceptance workflow

This notebook is meant to be a prototype for the NVIDIA NIM hackathon project.

## Installation and Requirements

Shits fucked cunt

In [ ]:
# Requirements
!pip install langchain==0.2.5
!pip install langchain_community==0.2.5
!pip install faiss-gpu # replace with faiss-gpu if you are using GPU
!pip install faiss-cpu
!pip install langchain-nvidia-ai-endpoints==0.1.2
!pip install requests pdfplumber spacy camelot-py 
!pip install pandas==2 numpy==1.26.4 
!pip install beautifulsoup4 
!pip install pymupdf
!pip install lxml
!pip install unstructured

## Getting Started!

You need an `NVIDIA_API_KEY` to use the NVIDIA API Catalog:

1) Create a free account with [NVIDIA](https://build.nvidia.com/explore/discover).
2) Click on your model of choice.
3) Under Input select the Python tab, and click **Get API Key** and then click **Generate Key**.
4) Copy and save the generated key as NVIDIA_API_KEY. From there, you should have access to the endpoints.
5) If at any point downstream NVIDIA complains about insufficient credits, tell them to shut the fuck up and make a new gmail. If their stock price is so high, they should share some of those profits with me, an NVIDIA investor (i own 3 shares rn)

Available NVIDIA api keys;

1. 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
2. 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

In [38]:
import getpass
import os

nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

## LLM & Embedding

### 1) Initialize the LLM

The ChatNVIDIA class is part of LangChain's integration (langchain_nvidia_ai_endpoints) with NVIDIA NIM microservices.
It allows access to NVIDIA NIM for chat applications, connecting to hosted or locally-deployed microservices.

Here we will use **mixtral-8x7b-instruct-v0.1**

In [33]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", max_tokens=1024)

# Can choose any model hosted at Nvidia API Catalog (Uncomment the below code to list the availabe models)
# ChatNVIDIA.get_available_models()

#### Note:
- In this notebook, we have used NVIDIA NIM microservices from the NVIDIA API Catalog.
- The other APIs, ChatNVIDIA, NVIDIAEmbedding, and NVIDIARerank, also support self-hosted NIM microservices.
- Change the `base_url` to your deployed NIM URL.
- Example: `llm = ChatNVIDIA(base_url="http://localhost:8000/v1", model="meta/llama3-8b-instruct")`
- NIM can be also hosted locally using Docker, following the [NVIDIA NIM for LLMs](https://docs.nvidia.com/nim/large-language-models/latest/getting-started.html) documentation. This is only true if you are the son of a rich oil tycoon, and have a few H100s sitting around in your basement.

In [12]:
# Example Code snippet if you want to use a self-hosted NIM
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# connect to an LLM NIM running at localhost:8000, specifying a specific model
llm = ChatNVIDIA(base_url="http://localhost:8000/v1", model="meta/llama3-8b-instruct")

### 2) Intialize the embedding
NVIDIAEmbeddings is a client to NVIDIA embeddings models that provides access to a NVIDIA NIM for embedding. It can connect to a hosted NIM or a local NIM using a base URL

We selected **NV-Embed-QA** as the embedding

In [34]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="NV-Embed-QA", truncate="END")

### 3) Obtain dataset
I love taxes and work! Lets steal information about taxes and work!

#### In the **DataHandler** class defined below, we can;

A) Walk through a webpage and find all sub-webpages and scrape the parent and children,

B) Extract texts, tables and images from pdfs

Real world documents can be very long, this makes it hard to fit in the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we’ll split the Document into chunks for embedding and vector storage. More on text splitting [here](https://python.langchain.com/v0.2/docs/concepts/#text-splitters)

In [49]:
import os
import requests
import pandas as pd
import urllib.parse  # To handle URL joining
import fitz

from tqdm import tqdm
from io import StringIO
from bs4 import BeautifulSoup, SoupStrainer
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, DataFrameLoader, CSVLoader, UnstructuredTSVLoader

class DataHandler:
    """
    Masterfully handles data scraping, preprocessing, and other data-related functionalities in this notebook.
    """

    def __init__(self,
                csv_path="./data/csv"):
        self.visited_urls = set() # classwide tracker to prevent repeated visits
        self.webloaders = [] # tracks all urls that have been converted to langchain WebBaseLoaders.
        self.tabular = [] # tracks all tabular data that has been discovered by scraper. Delivers a list of CSVLoader. Cowardly refusing to save to same list
        self.raws = []
        self.csv_path = csv_path

        self.text_splitter = CharacterTextSplitter(chunk_size=400, separator=" ", chunk_overlap=80)

        self.visited_urls = set()   # Classwide tracker to prevent repeated visits when scraping web
        self.tabular_data = []      # Tracks all tabular_data data that has been discovered by scraper. Delivers a list of CSVLoader. Cowardly refusing to save to same list.
        self.textual_data = []      # Tracks all textual data scraped from websites or pdfs
        self.all_data = []  # Just defining a joint list here instead of later during the functional call; in case we do not care about seperating them anymore (both are lists of Documents, just from different
        # base sources)

        if not os.path.exists(self.csv_path):
            os.mkdir(self.csv_path)
    
    @staticmethod
    def from_csv(csv_path):
        """
        Small func to read from csv and produce CSVLoaders.
        """
        df = pd.from_csv( csv_path) # so we can yoink its columns
        loader = CSVLoader(file_path=csv_path,
                                    csv_args={  'delimiter': ',',
                                                'quotechar': '"',
                                                'fieldnames': [str(col) for col in df.columns]}
                                    )
        return loader

    def csvs_to_loader(self, directory):
        """
        walks through directory, finds all csvs, and saves it into self.tabular 
        """
        for dir, subdir, files in os.walk(directory):
            for file in files:
                if file.endswith(".csv"):
                    fp = os.path.join(dir, file)
                    loader = self.from_csv(fp)
                    self.tabular.append(loader)


    def extract_table_elements(self, url, table_elements):
        """
        Helper func to extract tabular data

        Args:
            - url: url the table is under
            - table_elements: list of table elements
        """
        for table_idx, table in enumerate(table_elements): #TODO: Find better way to index different tables on the same page? not all have class attributes we can ID them with.
            try:
                tags = table.find_all('sup')
                for tag in tags:
                    tag.extract()

                # Attempt to find the title of this table. tableheader elements only tell us (pandas) how to index it, but what we need the header element for context on what this table is about
                tablename = f"table_{table_idx}" # default name presuming none is found
                for headertype in ['h3', 'h4']: # unlikely to lie in h2 or h1? could result in duplicate data. if we find by those.
                    header = table.find_previous(headertype)
                    if header is not None and header.text is not None: # find the closest header
                        tablename = header.text.replace("\n", "").replace("#", '').replace("\r", '').replace("/", '')
                        break
                
                tablename = os.path.basename(url) + " " + f"{tablename}" # what we will call this table, some has really annoying spacing, so maybe .replace(' ', '')?
                print(f"Grabbing table data under tablename {tablename}")
                df = pd.read_html(StringIO(str(table)), header=0)[0] # some tables do not have tableheader <th> tags for first row 
                # which would result in  generic column indices being created, so forcibly set first row as tableheader. 
                df['context'] = [tablename] * df.shape[0]

                if len(tablename) >= os.pathconf('/', 'PC_NAME_MAX'): # prevent shit from exploding because my tablename is damn scuffed but what todo.
                    tablename = tablename[:os.pathconf('/', 'PC_NAME_MAX') - 10]
                csv_path = os.path.join(self.csv_path, tablename + '.csv')
                # print(df)

                # print("csv_path:", csv_path)
                # print("csv_path:", csv_path)
                df.to_csv( csv_path, index=False ) # if true will fuck up columning in csvloader
                # loader = UnstructuredTSVLoader(csv_path, mode='elements')
                loader = CSVLoader(file_path=csv_path,
                                    csv_args={  'delimiter': ',',
                                                'quotechar': '"',
                                                'fieldnames': [str(col) for col in df.columns]}
                                    )
                for row in loader.load()[1:]: # first row is just column indexes, so void
                    self.tabular.append(
                        self.text_splitter.split_text(
                            self.clean_text(row.page_content)
                            )
                        )

            except BaseException as e:
                print(f"Unable to extract table data from url {url} with error {e}, passing!")


    def create_loaders(self):
        """
        Seperate method to create the loaders. Directly appends to textual_data attribute and calls extract_table_elements to handle tabular data.
        """
        for url, soupy_little_guy in self.raws:
            loader = WebBaseLoader(
                    web_paths=(url,),  # No URL fetching as we already have the HTML content
                    bs_kwargs={"parse_only": SoupStrainer(['main'])},
                )
            html_content = loader.load()
            for i in range(len(html_content)):
                self.textual_data.extend(
                    self.text_splitter.split_text(
                        self.clean_text(
                            html_content[i].page_content
                )))

            table_elements = soupy_little_guy.find_all("table") 
            self.extract_table_elements(url=url, table_elements=table_elements)


    def scrape_website(self, base_url, max_depth, depth=0):
        """
        Wraps around a nested function get_from_website, so when ran will define a new function that knows that the sauce base_url is.
        Scuffed? Yes.
        """

        def get_from_website(url, max_depth, depth):
            """
            Recursively scrape a website by visiting links starting from url. 
            Because this is a mostly I/O bound operation, we make a seperate method that actually creates the Loaders.

            Parameters:
                - url:              URL to start scraping from
                - depth:            Current recursion depth
                - max_depth:        Maximum recursion depth to avoid infinite loops

            Returns:
            - appends to self.raws, [url, BeautifulSoup object] created from response content.
            """
            if url in self.visited_urls or depth > max_depth:
                return

            try:
                response = requests.get(url)
                soupy_little_guy = BeautifulSoup(response.content, 'html.parser')
                if response.status_code != 200:
                    return print(f"Failed to retrieve {url}")
            except Exception as e:
                return print(f"Error accessing {url} with error: {e}")
                

            self.visited_urls.add(url)
            print("Current url:", url)
            self.raws.append([url, soupy_little_guy])

            for link in soupy_little_guy.find_all('a', href=True):  # Find all links on the current page
                relative_url = link['href']
                absolute_url = urllib.parse.urljoin(url, relative_url)
                if base_url in absolute_url:  # Avoids external sites
                    get_from_website(absolute_url, max_depth, depth + 1)

        get_from_website(base_url, max_depth, depth)


    def scrape_pdf(self, pdf_path):
        """
        Extracts information from pdf files.
        Texts stay as texts.
        Tables and images...

        Parameters:
            - pdf_path:     PDF file to extract from

        Output:
            - textual_data: List of cleaned strings that represent data from pdf doc
        """
        try:
            pdf_document = fitz.open(pdf_path)
        except:
            return print(f"Unable to open {pdf_path}")
        print(f"Current pdf: {pdf_path}")
        pdf_text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            pdf_text += page.get_text("text")

        self.textual_data.extend(
            self.text_splitter.split_text(
                self.clean_text(pdf_text)
        ))
    
    def clean_text(self, text):
        """
        Cleans text retrieved from sources to reduce the storage needed

        get_from_website(base_url, max_depth, depth)
        Parameters:
            - text:         Original string

        Output:
            - cleaned_text: Cleaned string 
        """
        return text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('  ', ' ')
    
    def embed_text(self, text):
        """
        Embeds a given text if necessary

        Parameters:
            - text:     Original string

        Output:
            - text:     Embedded string 
        """
        return embedder.embed_query(text)

We define our websites and pdf links below to call the dataloader on.

In [43]:
# Max depth determines whether you wish to look into children of parents websites, else set to 0
max_depth = 2
websites = [
    "https://www.iras.gov.sg/taxes",
    "https://www.iras.gov.sg/schemes",
    "https://www.mom.gov.sg/passes-and-permits",
    "https://www.mom.gov.sg/employment-practices",
    "https://www.mom.gov.sg/workplace-safety-and-health"
]

datahandler = DataHandler()
for website in websites:
    datahandler.scrape_website(website, max_depth)

if os.path.isdir('./data/pdfs'):
    for pdf in os.listdir('./data/pdfs'):
        datahandler.scrape_pdf(os.path.join('./data/pdfs', pdf))

Current url: https://www.iras.gov.sg/taxes
Current pdf: ./data/pdfs\verification-checklist-inspection-of-machines.pdf


In [44]:
datahandler.create_loaders() # only when we actually want to create loaders to debug stuff, so we don't need to scrape within the same function that we're (likely) debugging, saving time.

And then, we embed the textual and tabular data gathered for use in the vector db.

In [48]:
datahandler.embedded_data = [datahandler.embed_text(text) for text in datahandler.textual_data] + [datahandler.embed_text(text) for text in datahandler.tabular_data]
datahandler.textual_data.extend(datahandler.tabular_data)

Exception: [402] Payment Required
Account 'GIejbs1uP2wNdJHs1XWh8uHtgNwzs3BDRY4GFMZxNlw': Cloud credits expired - Please contact NVIDIA representatives

### 4) Storing the documents

To build our foundational knowledge base from our collected data and allow for faster retrieval of vector queries, we need to have some form of search system.

#### a) Process the documents into vectorstore and save it to disk

Vectorstores are good when we wish to store small datasets mainly in memory, thus using less storage.

In this case, we use FAISS, which is a high-performace library that is efficient for similarity search and clustering of dense vectors.

In [26]:
# Here we create a faiss vector store from the documents and save it to disk.
from langchain_community.vectorstores import FAISS

# You will only need to do this once, later on we will restore the already saved vectorstore
store = FAISS.from_texts(datahandler.embedded_data, embedder)
VECTOR_STORE = './data/nv_embedding'
store.save_local(VECTOR_STORE)

To enable runtime search, we index text chunks by embedding each document split and storing these embeddings in a vector database. Later to search, we embed the query and perform a similarity search to find the stored splits with embeddings most similar to the query.

Then, we can read the previously processed and saved vector store back for use;

In [27]:
from langchain_community.vectorstores import FAISS

# Load the FAISS vectorestore back.
VECTOR_STORE = './data/nv_embedding'
store = FAISS.load_local(VECTOR_STORE, embedder, allow_dangerous_deserialization=True)

#### b) Store all our data into a vectordb (connects vector stores to structured db systems)

We will be using **Milvus DB** since it is an open-source, distributed vector DB designed for high-performance vector similarity search across massive datasets (which is what we kind of will have), and makes use of popular libraries like FAISS or Annoy for its vector searching.

We have the option to use;
1) MilvusVectorStore (Milvus Lite) which is easier to implement and can be more easily tied with ML frameworks.
2) MilvusClient (Python SDK) WITHOUT Docker which was used in a demo to build a RAG system. --> What we shall try with first
3) Direct DB WITH Docker (Milvus Standalone) to directly manage and control the connections as well as increase flexibility.

Sources used:
- https://milvus.io/docs/quickstart.md
- https://milvus.io/docs/integrate_with_langchain.md
- https://milvus.io/docs/multimodal_rag_with_milvus.md
- https://github.com/milvus-io/bootcamp/tree/master/bootcamp/tutorials/quickstart/apps/multimodal_rag_with_milvus (Possible demo to use)

##### Note:

Previously, I thought of using **Weaviate** because it is a free, open-source, scalable and reliable vector database service with decent amounts of documentation online.

The Weaviate DB works more with Object-Oriented design rather than rows/columns and does not need to be structured like an SQL DB since it already has integrated models/pre-computed embeddings to handle vector embeddings.

However, I realised it **could not** be GPU accelerated during its vector retrieval/indexing.

Sources used:
- https://python.langchain.com/docs/integrations/vectorstores/weaviate/
- https://weaviate.io/developers/weaviate/client-libraries/python

In [6]:
!pip install langchain-milvus
!pip install pymilvus

In [46]:
from pymilvus import MilvusClient
from langchain_milvus import Milvus, Zilliz

class MilvusDB:
    """
    Not-so-masterfully handles the vector database using Milvus and all DB related functionality.
    """

    def __init__(self):
        self.client = None
        self.collection_name = None

    def set_up_db(self):
        """
        Starts up connection to Weaviate DB and creates schema if not already created

        Parameters: None

        Output:     None
        """
        self.client = MilvusClient(uri="./milvus.db")

    def create_collection(self, collection_name, dimensions):
        """
        Creates a new collection in the DB

        Parameters: 
            - collection_name:  Name of collection to make
            - dimensions:       Number of dimensions for vector data

        Output:     None
        """     
        if self.client.has_collection(collection_name):
            self.client.drop_collection(collection_name)

        self.client.create_collection(
            collection_name=collection_name,
            dimension=dimensions,
            metric_type="IP",  # Inner product distance
            consistency_level="Strong",  # Strong consistency level
        )
        self.collection_name = collection_name

    def insert_data(self, original, embedded):
        """
        Adds document and embedding object pairs to the DB collection 

        Parameters:
            - original:     Original documents
            - embedded:     Embedded documents

        Output: None
        """
        data = []

        for i, embedded_line in enumerate(tqdm(embedded, desc="Creating embeddings")):
            data.append({"id": i, "vector": embedded_line, "text": original[i]})

        self.client.insert(collection_name=self.collection_name, data=data)

    def retrieve_data(self, question):
        """
        Retrieves vector data from DB based on embedded question

        Parameters:
            - question:     Embedded question as a vector

        Output:
            - search_res:   Results of vector retrieval
        """
        search_res = self.client.search(
            collection_name=self.collection_name,
            data=[question],  
            limit=3,  # Return top 3 results
            search_params={"metric_type": "IP", "params": {}},  # Inner product distance
            output_fields=["text"],  # Return the text field
        )
        return search_res

In [47]:
database = MilvusDB()
database.create_collection("Documents", len(datahandler.embedded_data))
database.insert_data(datahandler.embedded_data)

AttributeError: 'DataHandler' object has no attribute 'embedded_data'

### 5) Using data to answer questions

With our stored embedded data, we can retrieve relevant vectors stored in our vectorstore/db to answer embedded questions.

#### a) Using vectorstore to answer

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

retriever = store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

# Langchain's LCEL(LangChain Expression Language) Runnable protocol is used to define the chain
# LCEL allows pipe together components and functions
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### b) Using vectordb to answer

In [52]:
def generate_response(question):
    question = embedder.embed_query(question)
    retrieved_data = database.retrieve_data(question)
    retrieved_texts = '\n\n'.join([hit.entity.get("text") for hit in retrieved_data])
    
    prompt = f" \
        Context from database:\n{retrieved_texts}\n\n\
        User's question: {question}\n\n\
        Please generate an informative response.\
    "
    return llm.chat(prompt)

# TODO: test if this works

Case 1: Irrelevant question

In [11]:
print(chain.invoke("What is component two hundred and twenty five million?"))

There is no information provided in the documents about a component or entity called "two hundred and twenty five million." The documents contain information about various financial transactions, mortgage and loan balances, estate duty calculations, and rules related to M&A allowance and motor vehicle expenses, but there is no mention of a "component" with a value of 225 million.


In [53]:
print(generate_response("What is component two hundred and twenty five million?"))

Case 2: Simple questions

In [ ]:
print(chain.invoke("How do i file taxes for my company?"))

Case 3: Complex questions

In [ ]:
print(chain.invoke("In the event my foriegn employee is injured at work, how do i report the incident and claim reparations?"))

Case 4: Realistic questions

In [13]:
chain.invoke("I am an employer whose firm qualifies for PWCs. From 2025 onwards, how much Co-Funding can I recieve from the government?")
# In its current state, the model does an ethan for this quite hard question, although it at least does retrieve some (few?) relevant PWCS info.

'From 2025 onwards, as an employer whose firm qualifies for PWCS (Productivity Works Credits Scheme), you can receive co-funding from the government of up to 50% for the first tier of wage increases and 15% to 30% for the second tier of wage increases. This co-funding support applies to wage increases given in qualifying year 2025 and onwards. The gross monthly wage ceiling for PWCS co-funding will be increased to $3,000 in qualifying years 2025 and 2026.\n\nPlease note that the specific rates and details of the co-funding support may be subject to changes or updates in the PWCS guidelines. It is recommended to consult the official guidelines or contact the relevant authorities for the most accurate and up-to-date information.'

### 6) Enhancing accuracy for single data sources

This example demonstrates how a re-ranking model can be used to combine retrieval results and improve accuracy during retrieval of documents.

Typically, reranking is a critical piece of high-accuracy, efficient retrieval pipelines. Generally, there are two important use cases:

- Combining results from multiple data sources
- Enhancing accuracy for single data sources

Here, we focus on demonstrating only the second use case. If you want to know more, check [here](https://github.com/langchain-ai/langchain-nvidia/blob/main/libs/ai-endpoints/docs/retrievers/nvidia_rerank.ipynb)

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIARerank
from langchain_core.runnables import RunnableParallel

# We will narrow the collection to 100 results and further narrow it to 10 with the reranker.
retriever = store.as_retriever(search_kwargs={'k':100}) # typically k will be 1000 for real world use-cases
ranker = NVIDIARerank(model='nv-rerank-qa-mistral-4b:1', top_n=10)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

reranker = lambda input: ranker.compress_documents(query=input['question'], documents=input['context'])

chain_with_ranker = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | {"context": reranker, "question": lambda input: input['question']}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
print(chain_with_ranker.invoke("How do i file taxes for my company?"))

Based on the provided documents, to file taxes for your company, you need to follow these steps:

1. Ensure that you are duly authorized by your company as an 'Approver' for Corporate Tax (Filing and Applications) in Corppass. You can refer to the step-by-step guides for assistance on Corppass setup.
2. Have your Singpass and your company’s Unique Entity Number (UEN)/ Entity ID ready.
3. Visit the mytax.iras.gov.sg website to file the Corporate Income Tax Return for your company.
4. If your company is filing Form C, you need to submit the financial statements/certified accounts and tax computation(s) for the relevant Year of Assessment (YA).
5. If your company meets the qualifying conditions to file Form C-S or Form C-S (Lite), you can choose to file the simplified version, Form C-S (Lite), if your company has an annual revenue of $200,000 or below.

You can also visit the Basic Guide to Corporate Income Tax for Companies page to get help with filing your company’s tax returns for the 

In [ ]:
print(chain_with_ranker.invoke("In the event my foriegn employee is injured at work, how do i report the incident and claim reparations?"))

Based on the provided documents, if your foreign employee is injured at work, you can report the incident and claim reparations under the Work Injury Compensation Act (WICA). Specifically, the documents mention that input tax can be claimed for work injury compensation insurance that is obligatory under WICA for both local and foreign employees performing manual work or non-manual work earning $2,600 or less a month.

To report the incident and make a claim, you can visit the Ministry of Manpower (MOM) webpage on WICA or contact MOM at +65 6438 5122. However, it is important to note that medical and accident insurance premiums for your staff are generally not allowable for input tax claims under the GST (General) Regulations, unless the insurance or payment of compensation is obligatory under WICA or under any collective agreement within the meaning of the Industrial Relations Act.

Therefore, it seems that you can report the incident and claim reparations under WICA, but you should ve